## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [16]:
import pandas as pd


FILE_PATH = "/Users/nataliapodziolkina/Documents/data/cookie_cats.csv"
data = pd.read_csv(FILE_PATH)   

print(data.head())
print(data.shape)


   userid  version  sum_gamerounds  retention_1  retention_7
0     116  gate_30               3        False        False
1     337  gate_30              38         True        False
2     377  gate_40             165         True        False
3     483  gate_40               1        False        False
4     488  gate_40             179         True         True
(90189, 5)


In [17]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Вхідні дані
p1 = 0.19     # базове утримання
p2 = 0.20     # очікуване після змін
alpha = 0.05  # рівень значущості
power = 0.8   # потужність тесту

# 1. Обчислення розміру ефекту (Cohen’s h)
effect_size = proportion_effectsize(p1, p2)

# 2. Розрахунок потрібного обсягу вибірки
analysis = NormalIndPower()
n_per_group = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, alternative='two-sided')

# 3. Сумарна кількість користувачів
total_n = n_per_group * 2

print(f"Розмір ефекту (Cohen's h): {effect_size:.4f}")
print(f"Необхідна кількість користувачів у кожній групі: {n_per_group:.0f}")
print(f"Сумарна кількість користувачів: {total_n:.0f}")



Розмір ефекту (Cohen's h): -0.0252
Необхідна кількість користувачів у кожній групі: 24638
Сумарна кількість користувачів: 49276


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [18]:

df = pd.read_csv('cookie_cats.csv')

# 2. середнє значення показника показник `retention_7`
retention_by_version = df.groupby('version')['retention_7'].mean()

print("Середнє утримання через 7 днів по версіях:")
print(retention_by_version)

Середнє утримання через 7 днів по версіях:
version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [19]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

# Підрахунок кількості "успіхів" (гравців, що повернулися) та загальної кількості у кожній групі
successes = df.groupby('version')['retention_7'].sum()
nobs = df.groupby('version')['retention_7'].count()

# Z-тест для двох пропорцій (двосторонній)
z_stat, p_val = proportions_ztest(successes, nobs, alternative='two-sided')

# Довірчі інтервали 95% для кожної групи
confint_control = proportion_confint(successes['gate_30'], nobs['gate_30'], alpha=0.05, method='normal')
confint_treatment = proportion_confint(successes['gate_40'], nobs['gate_40'], alpha=0.05, method='normal')

print(f"z statistic: {z_stat:.3f}")
print(f"p-value: {p_val:.4f}")
print(f"Довірчий інтервал 95% для групи control: [{confint_control[0]:.4f}, {confint_control[1]:.4f}]")
print(f"Довірчий інтервал 95% для групи treatment: [{confint_treatment[0]:.4f}, {confint_treatment[1]:.4f}]")


z statistic: 3.164
p-value: 0.0016
Довірчий інтервал 95% для групи control: [0.1866, 0.1938]
Довірчий інтервал 95% для групи treatment: [0.1785, 0.1855]


4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


1. Формулювання гіпотез

H₀ (нульова гіпотеза):
Версія гри та утримання на 7-й день незалежні.
(Переміщення воріт не впливає на те, чи повертається користувач.)

H₁ (альтернативна гіпотеза):
Версія гри та утримання залежать одна від одної.
(Переміщення воріт вплинуло на ймовірність повернення гравців.)

In [20]:

from scipy.stats import chi2_contingency

# Спостережувані частоти
contingency = pd.crosstab(df['version'], df['retention_7'])
print("Таблиця частот:")
print(contingency, "\n")

# Тест Хі-квадрат
chi2, p_value, dof, expected = chi2_contingency(contingency)

print(f"χ² statistic: {chi2:.4f}")
print(f"Degrees of freedom: {dof}")
print(f"p-value: {p_value:.4f}")


Таблиця частот:
retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279 

χ² statistic: 9.9591
Degrees of freedom: 1
p-value: 0.0016
